In [2]:
a360ai

<A360 AI Interface for project: Danish-test>

In [3]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [4]:
import warnings
warnings.filterwarnings('ignore')

### 1. Set Default Data Repo and load data

In [5]:
a360ai.set_default_datarepo("test")

In [6]:
a360ai.list_datasets()

,base_name,extension,size
1,Telco-Customer-Churn.csv,csv,977501.0
2,X.parquet,parquet,174867.0
3,model.onnx,onnx,5616.0
4,y.parquet,parquet,42876.0


In [7]:
X = a360ai.load_dataset("X.parquet")
y = a360ai.load_dataset("y.parquet")

### 2. Model training

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=5)

In [9]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#### Use a360ai MDK to track model experiments

In [10]:
my_model = a360ai.get_or_create_model(model_name=f"churn_pred_demo_tensorflow_2")

In [11]:
experiment = my_model.get_or_create_experiment(
    experiment_name = f"churn_pred_demo_tensorflow_2",
    model_flavor = "tensorflow",
    enable_drift_monitoring=True,
    train_features=X_train,
    train_target=y_train,
    feature_names=list(X.columns),
    data_exploration_file="/home/jovyan/01_exploratory-data-analysis.ipynb",
    data_preparation_file="/home/jovyan/02_data-preprocessing.ipynb",
    model_training_file="/home/jovyan/03_model-training.ipynb",
)


//-- Experiment Loaded --//
Model Name: churn_pred_demo_tensorflow_1
Experiment Name: churn_pred_demo_tensorflow_1
Final Run Id: bcdf2f40-a790-4152-98ed-cce27c38c61b
Model Flavor: tensorflow
Input Signature: ndarray: float64 (26,)
Output Signature: Churn big_integer
Data Exploration File: /home/jovyan/01_exploratory-data-analysis.ipynb
Data Preparation File: /home/jovyan/02_data-preprocessing.ipynb
Model Training File: /home/jovyan/03_model-training.ipynb
Drift Monitoring Enabled: True



In [12]:
experiments = my_model.list_experiments()
experiments

,id,bestRun,model_id,experiment_name,best_run_id,model_flavor,input_signature,output_signature,data_exploration_file,data_preparation_file,model_training_file,baseline,train_shape,model_name,updated_at
0,45fd50bb-a4d9-474d-b643-ada7dbce614e,"{'id': 'bcdf2f40-a790-4152-98ed-cce27c38c61b',...",e86dfaf6-18a2-4b52-9a93-91ca1910a95f,churn_pred_demo_tensorflow_1,bcdf2f40-a790-4152-98ed-cce27c38c61b,tensorflow,"ndarray: float64 (26,)",Churn big_integer,/home/jovyan/01_exploratory-data-analysis.ipynb,/home/jovyan/02_data-preprocessing.ipynb,/home/jovyan/03_model-training.ipynb,"{'gender': {'mean': -7.200166389035681e-17, 's...",[26],churn_pred_demo_tensorflow_1,2022-06-02 05:04


In [13]:
type(X_train)

numpy.ndarray

In [14]:
type(y_train)

pandas.core.frame.DataFrame

In [15]:
!pip install keras

In [20]:
type(y_train.values)

numpy.ndarray

In [21]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
# from torch.utils.data import Dataset, DataLoader
import tensorflow as tf

# train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
# train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

# test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
# test_dataset = test_dataset.batch(64)


model = Sequential()
model.add(Dense(4 , activation='relu'))
model.add(BatchNormalization())
model.add(Dense(5, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# for i in range(3):
#     model.fit(train_dataset, epochs=1)
#     metrics = {'test_score': accuracy_score(y_test,[0 if i <=0.5 else 1for i in model.predict(X_test).reshape(-1,1)]),
#         'train_score': accuracy_score(y_train,[0 if i <=0.5 else 1for i in model.predict(X_train).reshape(-1,1)])}
#     with experiment.run_experiment() as run:    
#         run.log_metrics(metrics)
#         run.log_model(model)

for epoch in range(10):
    
    model.train_on_batch(X_train,y_train.values)

    metrics = {'test_score': accuracy_score(y_test,[0 if i <=0.5 else 1for i in model.predict(X_test).reshape(-1,1)]),
            'train_score': accuracy_score(y_train,[0 if i <=0.5 else 1for i in model.predict(X_train).reshape(-1,1)])}
    with experiment.run_experiment() as run: 
        run.log_metrics(metrics)
        run.log_model(model)
        
        run.log_model("datatype")
        run_log_model(X.colums)

2022-06-02 05:06:16.172129: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-02 05:06:16.172158: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-06-02 05:06:17.247445: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-02 05:06:17.247477: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-02 05:06:17.247498: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (notebook-server-danish-tf-3ce35bc-0): /proc/driver/nvidia/version does not exist
2022-06-02 05:06:17.247675: I tensorflow/

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


2022-06-02 05:06:19.940401: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-06-02 05:06:19.940539: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-06-02 05:06:19.941759: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1144] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.007ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2022-06-02 05:06:19.986684: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-06-02 05:06:19.986800: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-06-02 05:06:19.991914: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1144] Optimization results for grappler item: graph_to_optimize
  constant_folding: Graph size after: 33 nodes (-22), 38 edges 

In [23]:
runs = experiment.list_runs()
runs

,id,dataset,artifact_paths,metric_test_score,metric_train_score,metadata_run_time
0,351082f2-6cf9-4bdd-91a8-a8f642c9be0a,c2be29ae-1fd6-46be-ab66-1b139ea00bab,[{'id': 'b6d56e45-6427-4abc-ba97-7016c9a56d4f'...,0.672353,0.712711,0.000017
1,06230d0c-9099-428b-8cb3-9da1b1266299,c2be29ae-1fd6-46be-ab66-1b139ea00bab,[{'id': 'c99376d2-9b2a-4ffe-983b-90dd252e741b'...,0.704335,0.736178,0.000010
2,45bd9fbd-000f-4faf-9f6d-bf5ded4aeca0,c2be29ae-1fd6-46be-ab66-1b139ea00bab,[{'id': '718c3105-97c2-4ed3-a353-99ad0e500965'...,0.731343,0.751111,0.000010
3,b355efc0-ae03-47c7-8f9a-687c46a0cc6f,c2be29ae-1fd6-46be-ab66-1b139ea00bab,[{'id': 'a2407085-78fc-4f55-9f04-b0b2d49b3c09'...,0.749822,0.768711,0.000009
4,410744dc-8281-4f60-86d2-bd2b36c31fc4,c2be29ae-1fd6-46be-ab66-1b139ea00bab,[{'id': 'f6f92741-f1c3-4a82-b32d-b99a6565d849'...,0.760483,0.781333,0.000009
5,3496765b-63c3-4a71-ac9e-19aba7aa229b,c2be29ae-1fd6-46be-ab66-1b139ea00bab,[{'id': '2750d410-b1fd-4fd3-ab98-9fe53c206a66'...,0.767591,0.786667,0.000009
6,9d399ab4-921f-4681-954d-94701e987ea2,c2be29ae-1fd6-46be-ab66-1b139ea00bab,[{'id': '2c7b9e42-9511-40b7-9bad-c84b2c8bbbcf'...,0.771855,0.793956,0.000010
7,2f0c160b-75eb-4951-a2b6-1588912bd630,c2be29ae-1fd6-46be-ab66-1b139ea00bab,[{'id': '3a09486c-808f-4713-8159-64e9a94d74ef'...,0.774698,0.794311,0.000009
8,bcdf2f40-a790-4152-98ed-cce27c38c61b,c2be29ae-1fd6-46be-ab66-1b139ea00bab,[{'id': 'abc51a4d-b4bd-4920-a204-38e9be7e6413'...,0.783227,0.798933,0.000028
9,f98c812a-778e-4b00-8c16-6fa7426b5e69,c2be29ae-1fd6-46be-ab66-1b139ea00bab,[{'id': '75150569-c2ef-4f2b-bdd3-dea59eef405c'...,0.782516,0.800356,0.000009


In [25]:
runs.sort_values('metric_test_score',ascending=False)

,id,dataset,artifact_paths,metric_test_score,metric_train_score,metadata_run_time
8,bcdf2f40-a790-4152-98ed-cce27c38c61b,c2be29ae-1fd6-46be-ab66-1b139ea00bab,[{'id': 'abc51a4d-b4bd-4920-a204-38e9be7e6413'...,0.783227,0.798933,0.000028
9,f98c812a-778e-4b00-8c16-6fa7426b5e69,c2be29ae-1fd6-46be-ab66-1b139ea00bab,[{'id': '75150569-c2ef-4f2b-bdd3-dea59eef405c'...,0.782516,0.800356,0.000009
7,2f0c160b-75eb-4951-a2b6-1588912bd630,c2be29ae-1fd6-46be-ab66-1b139ea00bab,[{'id': '3a09486c-808f-4713-8159-64e9a94d74ef'...,0.774698,0.794311,0.000009
6,9d399ab4-921f-4681-954d-94701e987ea2,c2be29ae-1fd6-46be-ab66-1b139ea00bab,[{'id': '2c7b9e42-9511-40b7-9bad-c84b2c8bbbcf'...,0.771855,0.793956,0.000010
5,3496765b-63c3-4a71-ac9e-19aba7aa229b,c2be29ae-1fd6-46be-ab66-1b139ea00bab,[{'id': '2750d410-b1fd-4fd3-ab98-9fe53c206a66'...,0.767591,0.786667,0.000009
4,410744dc-8281-4f60-86d2-bd2b36c31fc4,c2be29ae-1fd6-46be-ab66-1b139ea00bab,[{'id': 'f6f92741-f1c3-4a82-b32d-b99a6565d849'...,0.760483,0.781333,0.000009
3,b355efc0-ae03-47c7-8f9a-687c46a0cc6f,c2be29ae-1fd6-46be-ab66-1b139ea00bab,[{'id': 'a2407085-78fc-4f55-9f04-b0b2d49b3c09'...,0.749822,0.768711,0.000009
2,45bd9fbd-000f-4faf-9f6d-bf5ded4aeca0,c2be29ae-1fd6-46be-ab66-1b139ea00bab,[{'id': '718c3105-97c2-4ed3-a353-99ad0e500965'...,0.731343,0.751111,0.000010
1,06230d0c-9099-428b-8cb3-9da1b1266299,c2be29ae-1fd6-46be-ab66-1b139ea00bab,[{'id': 'c99376d2-9b2a-4ffe-983b-90dd252e741b'...,0.704335,0.736178,0.000010
0,351082f2-6cf9-4bdd-91a8-a8f642c9be0a,c2be29ae-1fd6-46be-ab66-1b139ea00bab,[{'id': 'b6d56e45-6427-4abc-ba97-7016c9a56d4f'...,0.672353,0.712711,0.000017


#### Get the best run from model experiments

In [26]:
best_run_id = runs.sort_values("metric_test_score", ascending=False).id.values[0]
best_run_id

'bcdf2f40-a790-4152-98ed-cce27c38c61b'

In [27]:
my_model.set_final_run(experiment, best_run_id)